In [64]:
!python --version

Python 3.7.13


In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [67]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [84]:
  !pip install optuna

     |████████████████████████████████| 308 kB 4.9 MB/s 
     |████████████████████████████████| 210 kB 55.9 MB/s 
     |████████████████████████████████| 81 kB 9.1 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 146 kB 73.7 MB/s 
     |████████████████████████████████| 113 kB 64.9 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=cd4ba2eba3686fde739ffb550ac821f1c4e581e315f27a692d912e449b983fb6
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [85]:
import optuna

In [68]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [69]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country


In [70]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,661504,0.432846,0.142929,3089.637185,2973.703815,6574.771380,0.136156,-0.000001,-0.000059,112.347147,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,661504,0.398761,0.092221,2170.544505,2054.465986,4306.240937,0.119993,-0.002001,-0.002011,143.554688,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,661794,0.375874,0.099977,2321.871533,2545.325000,4580.418732,0.104365,0.000002,0.000088,99.384014,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,661504,0.326701,0.056816,1366.306035,1736.646891,2562.717631,0.071181,-2.218249e-03,-0.001156,129.199219,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,661794,0.437906,0.005276,971.777391,1614.749295,1906.571121,0.031534,-8.984312e-06,-0.000014,143.554688,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,661504,0.457251,0.285521,2808.849567,2441.508002,5528.454222,0.097706,5.253005e-07,-0.000182,92.285156,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [71]:
cols = [c for c in train.columns]

In [72]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
train[cols] = stdsc.fit_transform(train[cols])
test[cols] = stdsc.transform(test[cols])

In [73]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,-0.648987,-1.134700,-1.172775,-0.967173,-1.282262,-0.778708,-1.049366,-0.701822,0.365950,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,-0.142344,0.703835,-1.919805,-1.723546,-1.199076,-1.699029,-1.720368,0.207379,0.356319,0.877118,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,-0.298576,0.939176,2.349185,0.841052,0.373789,0.601497,-0.148723,0.212788,0.200619,-0.948482,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [74]:
train = pd.concat([train,train_labels],axis=1)

In [75]:
train.head()

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var,label
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657,reggae
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347,reggae
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858,country
3,-0.142344,0.539357,0.680464,-0.445936,-0.631178,-0.478404,-0.333189,0.210407,0.310167,-0.948482,...,0.339539,-0.141871,0.382391,0.523603,-0.232210,-1.010304,-0.404582,-0.406441,-0.321155,blues
4,-0.142344,0.925586,-0.513290,0.087587,-0.601319,-0.250903,0.792784,-0.681551,-1.659057,-0.052279,...,-0.505731,-0.512941,0.509536,-0.097451,-0.645213,-0.679199,-1.395066,-0.948719,-1.377689,metal


In [76]:
N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 17171

In [ ]:
#rskf = RepeatedStratifiedKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=SEED)
#for fold, (trn_idx, val_idx) in enumerate(tqdm(rskf.split(X=train, y=train[target_column]), total=N_SPLITS*N_REPEATS)):
 ##   X_train, y_train = train[features].iloc[trn_idx], train[target_column].iloc[trn_idx]
  #  X_valid, y_valid = train[features].iloc[val_idx], train[target_column].iloc[val_idx]
   # X_test = test[features]

In [77]:
target_column = 'label'

In [78]:
features = [c for c in train.columns if c not in [target_column]]

In [79]:
len(features)

58

In [80]:
from sklearn.ensemble import RandomForestClassifier

In [81]:
X = train.drop('label',axis=1)
y=train['label']

In [82]:
X.shape, y.shape

((900, 58), (900,))

In [86]:
## going by the optuna way:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 101,2)
    min_samples_split = trial.suggest_float("min_samples_split", 0, 1)
    n_estimators = trial.suggest_int("n_estimators", 1, 999,2)
    max_depth = trial.suggest_int("max_depth", 1, 101,2)
    max_features = trial.suggest_categorical("max_features",["auto","sqrt"])
    rfc = RandomForestClassifier(min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_estimators=n_estimators, max_depth = max_depth, max_features = max_features)
        
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(rfc, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1,error_score='raise')
    score = score.mean()
    return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

[I 2022-04-22 11:25:44,748] A new study created in memory with name: no-name-7b445b98-ca35-40c6-aa18-602367c3385e
[I 2022-04-22 11:25:47,707] Trial 0 finished with value: 0.5318518518518519 and parameters: {'min_samples_leaf': 7, 'min_samples_split': 0.1850336100626757, 'n_estimators': 29, 'max_depth': 49, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.5318518518518519.
[I 2022-04-22 11:26:23,134] Trial 1 finished with value: 0.47370370370370374 and parameters: {'min_samples_leaf': 67, 'min_samples_split': 0.2973163018483065, 'n_estimators': 611, 'max_depth': 11, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.5318518518518519.
[I 2022-04-22 11:26:43,121] Trial 2 finished with value: 0.10555555555555558 and parameters: {'min_samples_leaf': 79, 'min_samples_split': 0.9872114106903184, 'n_estimators': 873, 'max_depth': 59, 'max_features': 'auto'}. Best is trial 0 with value: 0.5318518518518519.
[I 2022-04-22 11:26:58,683] Trial 3 finished with value: 0.10555555555555558 an

In [ ]:
trial = study.best_trial      
print('Accuracy: {}'.format(trial.value))   

Accuracy: 0.7537037037037037


In [ ]:
print("Best hyperparameters: {}".format(trial.params))

Best hyperparameters: {'min_samples_leaf': 3, 'min_samples_split': 0.010614118825852809, 'n_estimators': 75, 'max_depth': 69, 'max_features': 'sqrt'}


In [ ]:
  optuna.visualization.plot_optimization_history(study)


In [87]:
rfc_params = {'max_depth': 75,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 0.006776263852646938,
 'n_estimators': 413}

In [88]:
rfc_params  

{'max_depth': 75,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 0.006776263852646938,
 'n_estimators': 413}

In [89]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report


In [90]:
import statistics
from statistics import mean

In [91]:
%%time
N_SPLITS = 5
y_preds = []
acc= []
recall= []
f1= []
precision = []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = RandomForestClassifier(**rfc_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

  #print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))
  recall.append(recall_score(y_valid, valid_pred, average = 'micro'))
  f1.append(f1_score(y_valid, valid_pred, average = 'micro'))
  precision.append(precision_score(y_valid, valid_pred, average = 'micro'))

  y_preds.append(model.predict(test))

print('the mean accuracy is: {}'.format(mean(acc)))

print('the mean precision is: {}'.format(mean(precision)))
print('the mean micro recall is: {}'.format(mean(recall)))

print('the mean micro f1_score is: {}'.format(mean(f1)))

the mean accuracy is: 0.7788888888888889
the mean precision is: 0.7788888888888889
the mean micro recall is: 0.7788888888888889
the mean micro f1_score is: 0.778888888888889
CPU times: user 14.7 s, sys: 70.6 ms, total: 14.8 s
Wall time: 20.3 s


In [92]:
mean(acc)

0.7788888888888889

In [ ]:
y_preds

[array(['reggae', 'classical', 'hiphop', 'country', 'blues', 'rock', 'pop',
        'classical', 'blues', 'blues', 'disco', 'disco', 'metal', 'reggae',
        'jazz', 'country', 'country', 'reggae', 'jazz', 'blues', 'country',
        'hiphop', 'pop', 'classical', 'disco', 'country', 'disco', 'metal',
        'pop', 'jazz', 'jazz', 'classical', 'rock', 'classical', 'disco',
        'rock', 'classical', 'classical', 'pop', 'blues', 'rock', 'blues',
        'jazz', 'classical', 'hiphop', 'hiphop', 'jazz', 'blues', 'pop',
        'pop', 'disco', 'blues', 'hiphop', 'rock', 'jazz', 'pop', 'blues',
        'blues', 'reggae', 'reggae', 'blues', 'pop', 'pop', 'country',
        'classical', 'metal', 'classical', 'hiphop', 'hiphop', 'metal',
        'reggae', 'pop', 'metal', 'classical', 'classical', 'jazz',
        'hiphop', 'reggae', 'disco', 'reggae', 'rock', 'country', 'hiphop',
        'blues', 'classical', 'blues', 'blues', 'disco', 'rock', 'metal',
        'country', 'disco', 'metal', '

In [ ]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [ ]:
arr = [y_preds[0][0],y_preds[1][0], y_preds[2][0], y_preds[3][0], y_preds[4][0]]

In [ ]:
y_pred = pd.DataFrame(columns=['label'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [ ]:
pred

['reggae',
 'classical',
 'hiphop',
 'country',
 'blues',
 'rock',
 'pop',
 'classical',
 'blues',
 'blues',
 'disco',
 'disco',
 'metal',
 'reggae',
 'jazz',
 'country',
 'blues',
 'reggae',
 'jazz',
 'blues',
 'country',
 'hiphop',
 'pop',
 'classical',
 'disco',
 'country',
 'disco',
 'metal',
 'pop',
 'jazz',
 'jazz',
 'classical',
 'rock',
 'classical',
 'disco',
 'rock',
 'classical',
 'classical',
 'pop',
 'blues',
 'rock',
 'blues',
 'jazz',
 'classical',
 'hiphop',
 'hiphop',
 'jazz',
 'blues',
 'pop',
 'pop',
 'disco',
 'blues',
 'hiphop',
 'rock',
 'jazz',
 'pop',
 'blues',
 'blues',
 'reggae',
 'reggae',
 'blues',
 'pop',
 'pop',
 'country',
 'classical',
 'metal',
 'classical',
 'hiphop',
 'hiphop',
 'metal',
 'reggae',
 'pop',
 'metal',
 'classical',
 'classical',
 'jazz',
 'hiphop',
 'reggae',
 'disco',
 'reggae',
 'rock',
 'country',
 'hiphop',
 'blues',
 'classical',
 'blues',
 'blues',
 'disco',
 'rock',
 'metal',
 'country',
 'disco',
 'metal',
 'metal',
 'blues',
 '

In [ ]:
y_pred["label"]=pred

In [ ]:
y_pred

,label
0,reggae
1,classical
2,hiphop
3,country
4,blues
...,...
95,reggae
96,disco
97,pop
98,pop


In [ ]:
y_pred.to_csv('rfc_optuna_pred.csv', index=False)

In [ ]:
!cp -r '/content/rfc_optuna_pred.csv' /content/drive/MyDrive/project9/



In [ ]:
importances = RandomForestClassifier.feature_importances_
#
# Sort the feature importance in descending order
#
sorted_indices = np.argsort(importances)[::-1]
import plotly.graph_objs as go

import plotly.offline as py



In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(model.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ),
    margin=dict(
    l=300,
),
)

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

In [ ]:
train[['length','label']].head(15)

,length,label
0,-0.298576,reggae
1,-0.298576,reggae
2,-0.142344,country
3,-0.142344,blues
4,-0.142344,metal
5,-0.142344,blues
6,-0.142344,rock
7,-0.298576,metal
8,-0.142344,blues
9,-0.142344,jazz
